In [1]:
import os
import glob
import sys
sys.path.insert(0, os.path.join(os.getcwd(), 'icenet'))
import config
from tqdm import tqdm
import pandas as pd
import xarray as xr
import time
import numpy as np

In [ ]:
# Issue: nearly 50% of the dats missing before 1991
for year in range(1979, 1978, -1):
    for month in range(12, 0, -1):
        command = 'wget -r --user="anonymous" ftp://osisaf.met.no/reprocessed/ice/conc/v2p0/' + \
                                                    str(year) + '/' + str(month).zfill(2) + '/*nh*.nc'
        print(command)
        os.system(command)

In [ ]:
# Issue: 20-21/02/2022 and 09/11/2021 missing
for year in range(2023, 2015, -1):
    for month in range(12, 0, -1):
#       command = 'wget -r --user="anonymous" ftp://osisaf.met.no/reprocessed/ice/conc-cont-reproc/v2p0/' + \
        command = 'wget -r --user="anonymous" ftp://osisaf.met.no/reprocessed/ice/conc-cont-reproc/v3p0/' + \
                                                    str(year) + '/' + str(month).zfill(2) + '/*nh*.nc'
        print(command)
        os.system(command)

In [2]:
path = '/home/jovyan/icenet-paper/icenet/osisaf.met.no/reprocessed/ice/'

The computation has been done with nco & cdo operators:
```
#!/bin/bash
for year in {1979..2015}
#for year in {2016..2023}
do
   echo "$year"
   ncrcat ice/ice_conc_nh_ease2-250_cdr-v2p0_"$year"*.nc ice_conc_nh_ease2-250_cdr-v2p0_"$year".nc
   cdo ymonmean ice_conc_nh_ease2-250_cdr-v2p0_"$year".nc ice_conc_nh_ease2-250_cdr-v2p0_"$year"_monthly_mean.nc
done
```

In [ ]:
# Fix inconsistent latitude values after 2015 (names changed from icdr _> cdr)
path = '/home/jovyan/icenet-paper/icenet/osisaf.met.no/reprocessed/'
for year in range(2015, 2024, 1):
    name = 'ice_conc_nh_ease2-250_'
    if (year == 2015):
        filename = name + 'cdr-v2p0_' + str(year) + '_monthly_mean.nc'
        dset = xr.open_dataset(path + filename)
        latitudes = dset.lat.values  
    else:
        ifilename = name + 'icdr-v2p0_' + str(year) + '_monthly_mean.nc'
        dset = xr.open_dataset(path + ifilename)
        dset.lat.values = latitudes
        filename = name + 'cdr-v2p0_' + str(year) + '_monthly_mean.nc'
        dset.to_netcdf(path + filename)

In [5]:
path = '/home/jovyan/icenet-paper/icenet/osisaf.met.no/reprocessed/'
filename = 'ice_conc_nh_ease2-250_cdr-v2p0_*_monthly_mean.nc'
files =  glob.glob(os.path.join(path, filename))
files.sort()
files
da = xr.open_mfdataset(files)